In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from keras.applications import InceptionV3
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import ipywidgets as widgets
import io
from PIL import Image
import tqdm
from sklearn.model_selection import train_test_split
import cv2
from sklearn.utils import shuffle
import tensorflow as tf
import os


In [2]:

# Load data
X_train = []
Y_train = []
image_size = 150
labels = ['glioma_tumor','meningioma_tumor','no_tumor','pituitary_tumor']


In [3]:

for i in labels:
    folderPath = os.path.join('./Training', i)
    for j in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath, j))
        img = cv2.resize(img, (image_size, image_size))
        X_train.append(img)
        Y_train.append(i)

for i in labels:
    folderPath = os.path.join('./Testing', i)
    for j in os.listdir(folderPath):
        img = cv2.imread(os.path.join(folderPath, j))
        img = cv2.resize(img, (image_size, image_size))
        X_train.append(img)
        Y_train.append(i)


In [4]:

X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_train, Y_train = shuffle(X_train, Y_train, random_state=101)


In [5]:

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.1, random_state=101)


In [6]:

# Convert labels to categorical
y_train = tf.keras.utils.to_categorical([labels.index(i) for i in y_train])
y_test = tf.keras.utils.to_categorical([labels.index(i) for i in y_test])


In [7]:

# Load pre-trained InceptionV3 model without top (fully connected) layers
inception_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))




87910968/87910968 [==============================] - 24s 0us/step


In [8]:
# Freeze the convolutional base
for layer in inception_base.layers:
    layer.trainable = False



In [9]:
# Add custom top layers for classification
model = Sequential([
    inception_base,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')
])


In [10]:

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 512)               9437696   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 4)                 2

In [11]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])


C:\Anaconda\envs\myenv\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [12]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)


Epoch 1/20
75/75 [==============================] - 21s 100ms/step - loss: 11.3954 - accuracy: 0.4388 - val_loss: 1.9509 - val_accuracy: 0.5977
Epoch 2/20
75/75 [==============================] - 4s 60ms/step - loss: 2.0004 - accuracy: 0.5084 - val_loss: 0.9714 - val_accuracy: 0.6241
Epoch 3/20
75/75 [==============================] - 4s 60ms/step - loss: 1.1167 - accuracy: 0.5369 - val_loss: 1.0326 - val_accuracy: 0.6278
Epoch 4/20
75/75 [==============================] - 5s 60ms/step - loss: 1.0901 - accuracy: 0.5490 - val_loss: 1.0031 - val_accuracy: 0.5902
Epoch 5/20
75/75 [==============================] - 5s 60ms/step - loss: 1.0118 - accuracy: 0.5897 - val_loss: 0.9240 - val_accuracy: 0.6617
Epoch 6/20
75/75 [==============================] - 4s 59ms/step - loss: 0.9592 - accuracy: 0.5872 - val_loss: 0.8550 - val_accuracy: 0.6541
Epoch 7/20
75/75 [==============================] - 4s 60ms/step - loss: 0.9545 - accuracy: 0.5935 - val_loss: 0.8461 - val_accuracy: 0.6541
Epoch 8/20

In [13]:

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)


10/10 [==============================] - 2s 197ms/step - loss: 0.9259 - accuracy: 0.6475
Test Accuracy: 0.6474575996398926
